# 1. PREPARACIÓN NOTEBOOK

## 1.1 Librerías

In [19]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import openpyxl
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors
import logging
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors
import logging
from google.colab import drive
import sys
import os
import joblib
from IPython.display import display
from ipywidgets import interact
from sklearn.metrics.pairwise import cosine_similarity

## Carga y conexión

In [12]:
#Conectar al drive local
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
#Conectar al repositorio
path = '/content/drive/My Drive/cod/A3_marketing'

sys.path.append(path) #Importar las funciones propias a través de import, porque incluye la carpeta del repositorio como uno de esos paquetes para que import busque funciones
os.chdir(path) #Subir y descargar archivos de la ruta del repositorio de trabajo

In [14]:
%run a_funciones.ipynb

In [15]:
DATABASE_COPY_PATH = '/content/drive/My Drive/cod/A3_marketing/data/db_movies_c'
con, cur = conectar_bd(DATABASE_COPY_PATH)

Conexión exitosa a la base de datos (copia): /content/drive/My Drive/cod/A3_marketing/data/db_movies_c


In [16]:
output_path = '/content/drive/My Drive/cod/A3_marketing/salidas'
df_scaled = joblib.load(f"{output_path}/df_scaled.joblib")

In [17]:
ratings = pd.read_csv('/content/drive/My Drive/cod/A3_marketing/data/ratings.csv')

#2. DESPLIEGUE

In [24]:
# Crear carpeta si no existe
os.makedirs('/content/drive/My Drive/cod/A3_marketing/salidas/despliegue', exist_ok=True)

# Configuración del log
logging.basicConfig(
    filename='/content/drive/My Drive/cod/A3_marketing/salidas/despliegue/logs',  # Ruta del archivo log
    level=logging.INFO,  # Nivel de logging (INFO solo muestra mensajes básicos de ejecución)
    format='%(asctime)s - %(levelname)s - %(message)s'  # Formato del log: hora, nivel de log y mensaje
)

In [25]:
# --- Contenido (KNN general) ---
logging.info("Entrenando modelo KNN con datos escalados")
columnas_numericas = df_scaled.drop(columns=['title', 'movieId'])
model_knn = neighbors.NearestNeighbors(n_neighbors=11, metric='euclidean')
model_knn.fit(columnas_numericas)
_, idlist_knn = model_knn.kneighbors(columnas_numericas)
logging.info("Modelo KNN entrenado y vecinos más cercanos calculados")

# --- Filtro colaborativo por ítem ---
logging.info("Generando matriz usuario-película")
movie_user_matrix = ratings.pivot_table(index='movieId', columns='userId', values='rating').fillna(0)

logging.info("Calculando similitud coseno entre ítems")
item_similarity = cosine_similarity(movie_user_matrix)
item_similarity_df = pd.DataFrame(item_similarity, index=movie_user_matrix.index, columns=movie_user_matrix.index)
pelicula_dict = dict(zip(df_scaled['title'], df_scaled['movieId']))
logging.info("Matriz de similitud entre ítems creada")

# --- Funciones de recomendación ---
def recomendar_contenido(titulo):
    try:
        idx = df_scaled[df_scaled['title'] == titulo].index[0]
        logging.info(f"Recomendación por contenido para '{titulo}'")
        return [df_scaled.iloc[i].title for i in idlist_knn[idx][1:]]
    except Exception as e:
        logging.error(f"Error en recomendar_contenido('{titulo}'): {e}")
        return []

def recomendar_contenido_decada(titulo):
    try:
        idx = df_scaled[df_scaled['title'] == titulo].index[0]
        dec = df_scaled.loc[idx, 'decade']
        logging.info(f"Recomendación por contenido (misma década) para '{titulo}'")
        return [df_scaled.iloc[i].title for i in idlist_knn[idx][1:] if df_scaled.iloc[i].decade == dec]
    except Exception as e:
        logging.error(f"Error en recomendar_contenido_decada('{titulo}'): {e}")
        return []

def recomendar_colaborativo(titulo, n=10):
    try:
        movie_id = pelicula_dict.get(titulo)
        if movie_id not in item_similarity_df.index:
            logging.warning(f"Película '{titulo}' no encontrada en el sistema colaborativo.")
            return f"{titulo} no encontrado."
        similares = item_similarity_df[movie_id].sort_values(ascending=False)[1:n+1]
        logging.info(f"Recomendación colaborativa para '{titulo}'")
        return df_scaled[df_scaled['movieId'].isin(similares.index)][['title']].drop_duplicates()
    except Exception as e:
        logging.error(f"Error en recomendar_colaborativo('{titulo}'): {e}")
        return []

# --- Interfaz para demo
@interact
def demo_recomendador(titulo=df_scaled['title'].tolist()):
    print("\n🔹 Recomendaciones por contenido:")
    print(recomendar_contenido(titulo))

    print("\n🔹 Recomendaciones por contenido (misma década):")
    print(recomendar_contenido_decada(titulo))

    print("\n🔹 Recomendaciones colaborativas:")
    display(recomendar_colaborativo(titulo))

interactive(children=(Dropdown(description='titulo', options=('Toy Story', 'Jumanji', 'Grumpier Old Men', 'Fat…